# Image calibration, alignment and combination.

## Packages
First, import some basic packages:

In [1]:
import os.path
import glob
from pathlib import Path

import numpy as np
import pandas as pd

from astropy.io import fits

## Date, Object and filter definitions
These are used to define file paths.

In [2]:
obj = 'NGC5949'
filter = 'R'

# Instrument signature removal

Read in the calibration frames, subtract the bias from each science frame and divide by the flat. 

The flat is already bias-subtracted.

In [3]:
# Get list of raw science frames:
scifiles = glob.glob(os.path.join('processed',obj,filter,'raws','*.fits'))

# And check them:
scifiles

['processed/NGC5949/R/raws/r0743053.fits',
 'processed/NGC5949/R/raws/r0743054.fits',
 'processed/NGC5949/R/raws/r0743055.fits']

In [4]:
# First, create the directory to put the calibrated data in:
Path(os.path.join('processed',obj,filter,'postISR')).mkdir(parents=True, exist_ok=True)

# Read in the bias and flat frames:
bias = fits.open(os.path.join('calibs','bias2x2.fits'))
flat = fits.open(os.path.join('calibs',filter+'flatnorm2x2.fits'))

for scifile in scifiles:
    sci = fits.open(scifile)
    data = sci[0].data.astype('f8')
    header = sci[0].header
    gain = header['EGAIN']
    
    # Bias correct:
    data = data - bias[0].data
    data = data * gain
    
    # Create variance frame:
    var = np.copy(data)
    
    # Flat field
    data = data / flat[0].data

    # Correct for atmospheric extinction
    ext_per_airmass = 0.0547 # At 6550 Ang
    airmass = header['AIRMASS']
    mags_of_ext = ext_per_airmass * airmass
    intrinsic = data * (10.**(mags_of_ext / 2.5))

    # Write to files:
    newpath = os.path.join('processed',obj,filter,'postISR',os.path.basename(scifile))
    hdu_sci = fits.PrimaryHDU(data, header=header)
    hdu_var = fits.ImageHDU(var, header=header)
    hdul = fits.HDUList([hdu_sci, hdu_var])

    hdul.writeto(newpath, overwrite=True)

## Align the images
The images need to be aligned prior to being combined. To do this, you need to enter the (x,y) coordinates of the same star in each of your images into a file named xy.csv in your sci directory. The file should contain three columns, comma separated, and look like this (with exactly this header line):
| fname, x, y |
| --- |
| r0123456, 252, 328 |
| r0123457, 251, 329 |
| r0123458, 250, 330 |



In [5]:
# First, create the directory to put the aligned images in:
Path(os.path.join('processed',obj,filter,'shifted')).mkdir(parents=True, exist_ok=True)

# Read in (x,y) of star positions.
xys = pd.read_csv(os.path.join('xys',obj,filter,'xy.csv'))
fnames = xys['fname'].values
for i, fname in enumerate(fnames):
    hdu = fits.open(os.path.join('processed',obj,filter,'postISR',fname+'.fits'))
    header = hdu[0].header
    
    #Use the first frame in the list as a reference:
    sci = hdu[0].data.astype('f8')
    var = hdu[1].data.astype('f8')
    ind = fnames == fname
    
    if i == 0:
        xyref = xys[ind]
        xref = xyref['x'].values
        yref = xyref['y'].values
        
        hdu_sci = fits.PrimaryHDU(sci, header=header)
        hdu_var = fits.ImageHDU(var, header=header)
        hdul = fits.HDUList([hdu_sci, hdu_var])

    else:
        xyval = xys[ind]
        xval = xyval['x'].values
        yval = xyval['y'].values
    
        xdiff = xref - xval
        ydiff = yref - yval

        shiftedsci = np.roll(sci,xdiff, axis=1)
        shiftedsci = np.roll(shiftedsci,ydiff, axis=0)
        
        shiftedvar = np.roll(var,xdiff, axis=1)
        shiftedvar = np.roll(shiftedvar,ydiff, axis=0)
        
        hdu_sci = fits.PrimaryHDU(shiftedsci, header=header)
        hdu_var = fits.ImageHDU(shiftedvar, header=header)
        hdul = fits.HDUList([hdu_sci, hdu_var])
    
    newpath = os.path.join('processed',obj,filter,'shifted',fname+'.fits')
    hdul.writeto(newpath, overwrite=True)
    

## Median combine the images
We median-combine the image to get rid of hot pixels since, after alignment, they are unlikely to have the same x,y coordinates.

In [6]:
# First, create the directory to put the combined images in:
Path(os.path.join('processed',obj,filter,'combined')).mkdir(parents=True, exist_ok=True)

# Read in the aligned images
scifiles = glob.glob(os.path.join('processed',obj,filter,'shifted','*.fits'))
nfiles = len(scifiles)

#Create an array to hold the frames:
hdu = fits.open(scifiles[0])
sci = hdu[0].data.astype('f8')
header = hdu[0].header

shape = np.shape(sci)
scistack = np.zeros((shape[0],shape[1],nfiles))
varstack = np.zeros((shape[0],shape[1],nfiles))

for i,scifile in enumerate(scifiles):
    print(scifile)
    hdu = fits.open(scifile)
    
    sci = hdu[0].data.astype('f8')
    var = hdu[1].data.astype('f8')
    
    scistack[:,:,i] = sci
    varstack[:,:,i] = var
    
scicombined = np.median(scistack,2)
varcombined = np.median(varstack,2) / nfiles

hdu_sci = fits.PrimaryHDU(scicombined, header=header)
hdu_var = fits.ImageHDU(varcombined, header=header)
hdul = fits.HDUList([hdu_sci, hdu_var])

newpath = os.path.join('processed',obj,filter,'combined','combined.fits')
hdul.writeto(newpath, overwrite=True)

processed/NGC5949/R/shifted/r0743053.fits
processed/NGC5949/R/shifted/r0743054.fits
processed/NGC5949/R/shifted/r0743055.fits


In [7]:
himage = os.path.join('processed',obj,'H','combined')

In [10]:
mags_of_ext = 0.0547 * 1.72
print(20927 * 10.**(mags_of_ext / 2.5))

22821.310206385726
